In [44]:
import numpy as np
import pandas as pd
import glob
from os import path
import re
import seaborn as sns
import altair as alt
import matplotlib.pyplot as plt

sns.set_style("white")

In [45]:
# process results into csv
results = []
pattern = re.compile(r"(?P<func>(entropy_score|least_confidence|random_score|mcw|lc_mcw))_(?P<seed_size>\d+)_(?P<active_size>\d+)")
for fname in glob.glob("results_ablation/**"):
    match = re.search(pattern, fname)
    if match:
        results.append({
            "acquisition_function": match.group("func"),
            "seed_size": match.group("seed_size"),
            "active_batch_size": match.group("active_size"),
            "accuracy": np.load(fname)
        })
    else:
        print(f"skipping improperly formatted file: {fname}")
results = pd.DataFrame(results)
results.to_csv("results_ablation/results_ablation.csv", index=False)    

skipping improperly formatted file: results_ablation\cnn_active_learning_val_accuracy_tweet_count_10000_10.npy
skipping improperly formatted file: results_ablation\cnn_active_learning_val_accuracy_tweet_count_10000_25.npy
skipping improperly formatted file: results_ablation\cnn_active_learning_val_accuracy_tweet_count_10000_50.npy
skipping improperly formatted file: results_ablation\cnn_active_learning_val_accuracy_tweet_count_15000_10.npy
skipping improperly formatted file: results_ablation\cnn_active_learning_val_accuracy_tweet_count_15000_25.npy
skipping improperly formatted file: results_ablation\cnn_active_learning_val_accuracy_tweet_count_15000_50.npy
skipping improperly formatted file: results_ablation\cnn_active_learning_val_accuracy_tweet_count_20000_10.npy
skipping improperly formatted file: results_ablation\cnn_active_learning_val_accuracy_tweet_count_20000_25.npy
skipping improperly formatted file: results_ablation\cnn_active_learning_val_accuracy_tweet_count_20000_50.npy
s

In [46]:
results.dtypes

acquisition_function    object
seed_size               object
active_batch_size       object
accuracy                object
dtype: object

In [47]:
results["final_accuracy"] = results["accuracy"].apply(lambda l: l[-1])
results["seed_size"] = results["seed_size"].astype(int)
results["active_batch_size"] = results["active_batch_size"].astype(int)
max_accuracy_idx = results["final_accuracy"].argmax()
print(max_accuracy_idx)
print(results["accuracy"][max_accuracy_idx])
print(results["acquisition_function"][max_accuracy_idx])
print(results["seed_size"][max_accuracy_idx])
print(results["active_batch_size"][max_accuracy_idx])

32
[0.49194444 0.51333333 0.57791667 0.59125    0.59263889 0.60888889
 0.61527778 0.665     ]
least_confidence
20000
50


In [80]:
def plot_against_base(base_chart: alt.Chart):
    scatter = base.mark_boxplot().encode(
        y=alt.Y("final_accuracy:Q", scale=exclude_zero_point, title="Accuracy"),
        # color="acquisition_function:N"
    )
    line = base.mark_line().encode(
        y=alt.Y("mean(final_accuracy):Q", scale=exclude_zero_point, title="Mean accuracy"),
    )
    return (scatter + line).facet(column="acquisition_function:N")

In [67]:
# All else equal, what is the relationship between seed size and accuracy?
exclude_zero_point = alt.Scale(zero=False)


results_lc = results[results['acquisition_function'] == 'least_confidence']
results_ent = results[results['acquisition_function'] == 'entropy_score']
results_mcw = results[results['acquisition_function'] == 'mcw']
results_lcmcw = results[results['acquisition_function'] == 'lc_mcw' ]
results_rand = results[results['acquisition_function'] == 'random_score' ]


# base = alt.Chart(results[]).encode(
#     x=alt.X("seed_size:Q", scale=exclude_zero_point, title="Seed dataset size"),
    
# )
# plot_against_base(base)

# regression = base.encode(
#     y=alt.Y("mean(final_accuracy):Q", scale=exclude_zero_point)
# ).transform_regression(
#     'seed_size', 
#     'final_accuracy', 
#     groupby=['acquisition_function'],
#     extent=[results['seed_size'].min() - 1000, results['seed_size'].max() + 1000]
# ).mark_line(
#     color="orange", 
#     strokeDash=[3,3],
# )




In [82]:
result_temp = results[results['acquisition_function'] !='mcw']
base = alt.Chart(results_temp).encode(
    x=alt.X("seed_size:Q", scale=exclude_zero_point, title="Seed dataset size"),
    
)
plot_against_base(base)

ValueError: Objects with "config" attribute cannot be used within LayerChart. Consider defining the config attribute in the LayerChart object instead.

In [68]:

base = alt.Chart(results_lc).encode(
    x=alt.X("seed_size:Q", scale=exclude_zero_point, title="Seed dataset size"),
    
)
plot_against_base(base)

alt.FacetChart(...)

In [53]:

base = alt.Chart(results_ent).encode(
    x=alt.X("seed_size:Q", scale=exclude_zero_point, title="Seed dataset size"),
    
)
plot_against_base(base)

alt.FacetChart(...)

In [69]:

base = alt.Chart(results_mcw).encode(
    x=alt.X("seed_size:Q", scale=exclude_zero_point, title="Seed dataset size"),
    
)
plot_against_base(base)

alt.FacetChart(...)

In [70]:

base = alt.Chart(results_lcmcw).encode(
    x=alt.X("seed_size:Q", scale=exclude_zero_point, title="Seed dataset size"),
    
)
plot_against_base(base)

alt.FacetChart(...)

In [71]:

base = alt.Chart(results_rand).encode(
    x=alt.X("seed_size:Q", scale=exclude_zero_point, title="Seed dataset size"),
    
)
plot_against_base(base)

alt.FacetChart(...)

- Actual acquisition functions show a strong positive correlation b/w seed dataset size and accuracy. This makes sense because we expect more data to improve performance.
- This effect is much less strong in the random score function. *why?*

In [76]:
# All else equal, what is the relationship between active learning batch size and accuracy?
results_temp = results[results['acquisition_function'] !='mcw']
base = alt.Chart(results_temp).encode(
    x=alt.X("active_batch_size:Q", scale=exclude_zero_point, title="Active learning batch size"),   
)

plot_against_base(base)

alt.FacetChart(...)

In [61]:
base = alt.Chart(results_lc).encode(
    x=alt.X("active_batch_size:Q", scale=exclude_zero_point, title="Active learning batch size"),   
)

plot_against_base(base)

alt.FacetChart(...)

In [73]:
base = alt.Chart(results_ent).encode(
    x=alt.X("active_batch_size:Q", scale=exclude_zero_point, title="Active learning batch size"),   
)

plot_against_base(base)

alt.FacetChart(...)

In [74]:
base = alt.Chart(results_mcw).encode(
    x=alt.X("active_batch_size:Q", scale=exclude_zero_point, title="Active learning batch size"),   
)

plot_against_base(base)

alt.FacetChart(...)

In [75]:
base = alt.Chart(results_rand).encode(
    x=alt.X("active_batch_size:Q", scale=exclude_zero_point, title="Active learning batch size"),   
)

plot_against_base(base)

alt.FacetChart(...)

In [66]:
base = alt.Chart(results_lcmcw).encode(
    x=alt.X("active_batch_size:Q", scale=exclude_zero_point, title="Active learning batch size"),   
)

plot_against_base(base)

alt.FacetChart(...)

- Relationship seems much weaker overall
- if the relationship is significantly stronger for the actual functions than for the random function, that means our choice of functions is actually helping

In [42]:
# pearson's R per acquisition function
a = results.groupby("acquisition_function").corr()
b = a['final_accuracy']
b
a

seed_size  active_batch_size  \
acquisition_function                                                   
entropy_score        seed_size           1.000000           0.000000   
                     active_batch_size   0.000000           1.000000   
                     final_accuracy      0.845683           0.207130   
lc_mcw               seed_size           1.000000           0.000000   
                     active_batch_size   0.000000           1.000000   
                     final_accuracy      0.891845           0.304280   
least_confidence     seed_size           1.000000           0.000000   
                     active_batch_size   0.000000           1.000000   
                     final_accuracy      0.889154           0.203143   
random_score         seed_size           1.000000           0.000000   
                     active_batch_size   0.000000           1.000000   
                     final_accuracy      0.337486          -0.086663   
tweet_count          seed_size           1.000000           0.000000   
                     active_batch_size   0.000000           1.000000   
                     final_accuracy      0.536144           0.555862   

                                        final_accuracy  
acquisition_function                                    
entropy_score        seed_size                0.845683  
                     active_batch_size        0.207130  
                     final_accuracy           1.000000  
lc_mcw               seed_size                0.891845  
                     active_batch_size        0.304280  
                     final_accuracy           1.000000  
least_confidence     seed_size                0.889154  
                     active_batch_size        0.203143  
                     final_accuracy           1.000000  
random_score         seed_size                0.337486  
                     active_batch_size       -0.086663  
                     final_accuracy           1.000000  
tweet_count          seed_size                0.536144  
                     active_batch_size        0.555862  
                     final_accuracy           1.000000

In [16]:
latex_tab = a.to_latex(caption="Correlation Coefficients of Parameters and Final Accuracy")
print(latex_tab)

\begin{table}
\centering
\caption{Correlation Coefficients of Parameters and Final Accuracy}
\begin{tabular}{llrrr}
\toprule
            &                &  seed\_size &  active\_batch\_size &  final\_accuracy \\
acquisition\_function & {} &            &                    &                 \\
\midrule
entropy\_score & seed\_size &   1.000000 &           0.000000 &        0.845683 \\
            & active\_batch\_size &   0.000000 &           1.000000 &        0.207130 \\
            & final\_accuracy &   0.845683 &           0.207130 &        1.000000 \\
least\_confidence & seed\_size &   1.000000 &           0.000000 &        0.889154 \\
            & active\_batch\_size &   0.000000 &           1.000000 &        0.203143 \\
            & final\_accuracy &   0.889154 &           0.203143 &        1.000000 \\
random\_score & seed\_size &   1.000000 &           0.000000 &        0.337486 \\
            & active\_batch\_size &   0.000000 &           1.000000 &       -0.086663 \\
        

In [43]:
latex = b.to_latex(caption="Correlation Coefficients of Parameters and Final Accuracy")
print(latex)

\begin{table}
\centering
\caption{Correlation Coefficients of Parameters and Final Accuracy}
\begin{tabular}{llr}
\toprule
            &                &  final\_accuracy \\
acquisition\_function & {} &                 \\
\midrule
entropy\_score & seed\_size &        0.845683 \\
            & active\_batch\_size &        0.207130 \\
            & final\_accuracy &        1.000000 \\
lc\_mcw & seed\_size &        0.891845 \\
            & active\_batch\_size &        0.304280 \\
            & final\_accuracy &        1.000000 \\
least\_confidence & seed\_size &        0.889154 \\
            & active\_batch\_size &        0.203143 \\
            & final\_accuracy &        1.000000 \\
random\_score & seed\_size &        0.337486 \\
            & active\_batch\_size &       -0.086663 \\
            & final\_accuracy &        1.000000 \\
tweet\_count & seed\_size &        0.536144 \\
            & active\_batch\_size &        0.555862 \\
            & final\_accuracy &        1.000000 \\
\